In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.2MB 123kB/s 
     |████████████████████████████████| 421.8MB 33kB/s 
     |████████████████████████████████| 6.7MB 40.9MB/s 
     |████████████████████████████████| 983kB 40.0MB/s 
     |████████████████████████████████| 245kB 43.1MB/s 
     |████████████████████████████████| 675kB 30.7MB/s 
     |████████████████████████████████| 471kB 36.0MB/s 
     |████████████████████████████████| 3.9MB 40.7MB/s 
     |████████████████████████████████| 450kB 42.5MB/s 
     |████████████████████████████████| 1.1MB 35.9MB/s 
     |████████████████████████████████| 3.8MB 39.8MB/s 
     |████████████████████████████████| 890kB 26.4MB/s 
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
train_b=pd.read_csv("/content/drive/My Drive/minor/hindi_dataset/hindi_dataset.tsv",sep='\t',encoding='utf-8') #training data
train_b=train_b[train_b.task_2!='NONE']
test_b=pd.read_csv("/content/drive/My Drive/minor/hindi_dataset/hasoc2019_hi_test_gold_2919.tsv",sep='\t',encoding='utf-8') #testing data.
test_b=test_b[test_b.task_2!='NONE']
x_train,x_test,y_train,y_test=train_test_split(train_b['text'],train_b['task_2'],test_size=0.15,random_state=42) #training and validation split
target_names=['OFFN','PRFN','HATE'] #names of classes

In [6]:
#convertig the tweets to list of strings and labels to list data type
x_train=list(x_train)
x_test=list(x_test)
y_train=list(y_train)
y_test=list(y_test)

In [7]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-multilingual-cased' #bert predefined model
t = text.Transformer(MODEL_NAME, maxlen=500, classes=target_names) #load model, text and label preprocessor for bert
trn = t.preprocess_train(x_train, y_train) #preprocess text and labels for training data
val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data
model = t.get_classifier() #get bert classifier
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6) #get trainable learner for bert

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:398: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: hi
train sequence lengths:
	mean : 28
	95percentile : 57
	99percentile : 62


/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:520: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: hi
test sequence lengths:
	mean : 29
	95percentile : 57
	99percentile : 61


In [8]:
learner.fit_onecycle(5e-5, 4) #training for epochs=4 ,learning rate=5e-5



begin training using onecycle policy with max lr of 5e-05...
Train for 350 steps, validate for 12 steps
Epoch 1/4
350/350 [==============================] - 187s 535ms/step - loss: 0.9231 - accuracy: 0.5496 - val_loss: 0.8164 - val_accuracy: 0.6173
Epoch 2/4
350/350 [==============================] - 168s 481ms/step - loss: 0.7702 - accuracy: 0.6301 - val_loss: 0.7639 - val_accuracy: 0.6550
Epoch 3/4
350/350 [==============================] - 169s 482ms/step - loss: 0.7487 - accuracy: 0.6263 - val_loss: 0.7390 - val_accuracy: 0.6307
Epoch 4/4
350/350 [==============================] - 168s 481ms/step - loss: 0.6302 - accuracy: 0.7088 - val_loss: 0.7452 - val_accuracy: 0.6765


In [9]:
learner.validate() #predictions metrics for validation data

              precision    recall  f1-score   support

           0       0.50      0.42      0.46        88
           1       0.50      0.56      0.53       101
           2       0.85      0.86      0.86       182

    accuracy                           0.68       371
   macro avg       0.62      0.62      0.62       371
weighted avg       0.67      0.68      0.67       371



array([[ 37,  40,  11],
       [ 28,  57,  16],
       [  9,  16, 157]])

In [10]:
predictor = ktrain.get_predictor(learner.model, preproc=t) #get predictor for the trained model of  bert

In [11]:
predictor.get_classes()

['HATE', 'OFFN', 'PRFN']

In [17]:
test_b_text=list(test_b['text']) #get text for testing
test_b['task_2']=test_b['task_2'].map({'HATE':0,'OFFN':1,'PRFN':2})
test_b_label=list(test_b['task_2']) #get label of testing
# val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data

In [18]:
# predicted probability scores for each category
pred_y=predictor.predict_proba(test_b_text)

In [19]:
pred_y=np.argmax(pred_y,axis=1) #get class with maximum probability

In [20]:
from sklearn.metrics import classification_report

In [21]:
print(classification_report(test_b_label,pred_y)) # classification report

              precision    recall  f1-score   support

           0       0.58      0.52      0.54       190
           1       0.58      0.48      0.53       197
           2       0.73      0.92      0.81       218

    accuracy                           0.65       605
   macro avg       0.63      0.64      0.63       605
weighted avg       0.63      0.65      0.63       605

